## 1. Settings
Set config         : *("lr", "wd", "p_dims", ...)*  
Define scheduler   : *(ASHAScheduler, ...)*  

In [ ]:
import ray
from ray import tune

In [ ]:
from ray.tune.schedulers import ASHAScheduler

config={
    "lr": tune.loguniform(1e-5, 1e-3),
    "wd": tune.uniform(0, 0.1),
    # "anneal_cap": tune.uniform(0.1, 0.4),
    "p_dims_0": tune.choice([150,200,250,300]),
    "p_dims_1": tune.choice([500,600,700,800])
}
asha = ASHAScheduler(
    time_attr='training_iteration',
    max_t=200,
    grace_period=10,
    reduction_factor=3,
    brackets=3)

## 2. Run ray tune
import train code (you should not implement model & train cdoes at ipython)  
initialize ray `ray.init`  
`tune.run`  

In [ ]:
from train_multiaae import train_multiaae

In [ ]:
ray.shutdown()

In [ ]:
ray.init(num_cpus=5, num_gpus=1)

In [ ]:
result = tune.run(
    tune.with_parameters(train_multiaae),
    local_dir='/opt/ml/input/melon/results/',
    resources_per_trial={"cpu": 5, "gpu": 1},
    config=config,
    metric="best_n100",
    mode="max",
    num_samples=60,
    scheduler=asha
)

## 3. Load Results

In [20]:
from ray.tune import ExperimentAnalysis
analysis = ExperimentAnalysis("results/train_multiaae_2022-03-26_15-13-06")
best_trial = analysis.get_best_trial(metric="best_n100", mode="max")

f = open("best_config_multiaae.txt", 'w')
f.write("{}\n\n".format(analysis.get_best_trial(metric="best_n100", mode="max")))
f.write("{}".format(best_trial.config))
f.close()

2022-03-26 18:09:43,279	INFO experiment_analysis.py:696 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.
